H1 User Competency (Rasch Model)

In [1]:
import pandas as pd
import pickle
from scipy import stats
import numpy as np
import seaborn as sns
import pingouin as pg
import statsmodels.formula.api as s

In [2]:
#import
df = pickle.load(open( '../00_data/cleaned/sitzungssummary.pkl','rb') )
xmlsaetze = pickle.load(open( '../00_data/cleaned/xmlsaetze.pkl','rb') )

rasch = xmlsaetze[['UserID','SatzID','Erfolg','Datum']]

rasch['UserID'].astype(str)
rasch['SatzID'].astype(str)

c:\Python310\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


72780         4094
72783         4076
72784          373
72786         4097
72791         4100
            ...   
4981005    9991780
4981007    9991753
4981009    9991756
4981012    9991774
4981014    9991753
Name: SatzID, Length: 962598, dtype: object

In [3]:
test = rasch.groupby(['UserID','SatzID']).agg({'Datum':np.max})
test = test.reset_index()
test = pd.merge(test, rasch, on=['UserID','SatzID','Datum'])

In [5]:
values = [9991775, 373 ,375 ,379 ,382 ,385 ,386 ,387 ,400 ,3894 ,3895 ,3896 ,3897 ,3898 ,3899 ,3900 ,3901 ,3902 ,3903 ,3904 ,3905 ,3906 ,3907 ,3910 ,3911 ,3912 ,3913 ,3914 ,3916 ,3917 ,3918 ,3920 ,3922 ,3931 ,3937 ,3939 ,3948 ,3949 ,3980 ,4073 ,9991674 ,9991683 ,9991685 ,9991686 ,9991687 ,9991688 ,9991691 ,9991692 ,9991693 ,9991699 ,9991702 ,9991703 ,9991705 ,9991706 ,9991709 ,9991710 ,9991711 ,9991712 ,9991713 ,9991714 ,9991715 ,9991716 ,9991718 ,9991719 ,9991720 ,9991777 ,9991778 ,9991889 ,9991891 ,9991898 ,9991899 ,9992081 ,9992082 ,9992083 ,9992084 ,9992085 ,9992087 ,9992089 ,9992090 ,9992091 ,9992092 ,9992093 ,9992094 ,9992095 ,9992096 ,9992097 ,9992098 ,9992100 ,9992101 ,9992102 ,9992104 ,9992105 ,9992106 ,9992107 ,9992108 ,9992109 ,9992229 ,9992234 ,9992240 ,9992242 ,9992246 ,9992247 ,9992248 ,9992250 ,9992251 ,9992253 ,9992254 ,9992255 ,9992256 ,9992257 ,9992993 ,9992995 ,9992997]

#drop rows that contain any value in the list
test = test[test.SatzID.isin(values) == False]


In [6]:
test

,UserID,SatzID,Datum,Erfolg
0,186923,873,2022-09-06 10:20:40,1
1,186923,875,2022-09-06 10:17:42,1
2,186923,878,2022-09-06 10:21:23,1
3,186923,881,2022-09-06 10:20:21,1
4,186923,882,2022-09-06 10:21:13,1
...,...,...,...,...
517001,1064116,4087,2022-10-30 21:01:15,1
517002,1064116,4088,2022-10-30 20:59:14,1
517003,1064116,4094,2022-10-30 20:59:58,1
517004,1064116,4097,2022-10-30 21:00:05,1


In [11]:
final = pickle.load(open( '../00_data/final/competence.pkl','rb') )
final = final[['UserID','interventiongroup']]

In [13]:
df = pd.merge(test,final, on='UserID')
df

,UserID,SatzID,Datum,Erfolg,interventiongroup
0,186923,873,2022-09-06 10:20:40,1,6
1,186923,875,2022-09-06 10:17:42,1,6
2,186923,878,2022-09-06 10:21:23,1,6
3,186923,881,2022-09-06 10:20:21,1,6
4,186923,882,2022-09-06 10:21:13,1,6
...,...,...,...,...,...
502487,1064116,4087,2022-10-30 21:01:15,1,6
502488,1064116,4088,2022-10-30 20:59:14,1,6
502489,1064116,4094,2022-10-30 20:59:58,1,6
502490,1064116,4097,2022-10-30 21:00:05,1,6


In [26]:
df_grouped = df.groupby(['UserID','interventiongroup']).agg({'Erfolg':'sum','SatzID':'count'})
df_grouped['fehlerquotient'] = 1-(df_grouped.Erfolg / df_grouped.SatzID)
final = df_grouped
final = final.reset_index()
final

,UserID,interventiongroup,Erfolg,SatzID,fehlerquotient
0,186923,6,15,16,0.0625
1,360974,2,14,14,0.0000
2,373836,2,11,11,0.0000
3,409463,control,9,9,0.0000
4,435839,control,16,16,0.0000
...,...,...,...,...,...
11380,1063303,4,14,14,0.0000
11381,1063461,6,13,13,0.0000
11382,1063470,2,22,22,0.0000
11383,1063918,6,40,40,0.0000


In [27]:
intvcontrol = final[(final['interventiongroup']=='control')]
intv2 = final[(final['interventiongroup']=='2')]
intv3 = final[(final['interventiongroup']=='3')]
intv4 = final[(final['interventiongroup']=='4')]
intv5 = final[(final['interventiongroup']=='5')]
intv6 = final[(final['interventiongroup']=='6')]


In [31]:
prop_fehler_by_group = final.groupby(['interventiongroup']).agg({'fehlerquotient':'mean','UserID':'count'})
prop_fehler_by_group = prop_fehler_by_group.rename(columns={'UserID':'count'})
prop_fehler_by_group

,fehlerquotient,count
interventiongroup,,
2,0.057721,1786
3,0.055718,1880
4,0.059031,1861
5,0.107293,1766
6,0.058534,1684
control,0.056240,2408


In [29]:
#Wilcoxon Mann Whitney Test
print(stats.mannwhitneyu(intvcontrol['fehlerquotient'], intv2['fehlerquotient']))
print(stats.mannwhitneyu(intvcontrol['fehlerquotient'], intv3['fehlerquotient']))
print(stats.mannwhitneyu(intvcontrol['fehlerquotient'], intv4['fehlerquotient']))
print(stats.mannwhitneyu(intvcontrol['fehlerquotient'], intv5['fehlerquotient']))
print(stats.mannwhitneyu(intvcontrol['fehlerquotient'], intv6['fehlerquotient']))


MannwhitneyuResult(statistic=2180253.5, pvalue=0.3850049918342616)
MannwhitneyuResult(statistic=2283327.5, pvalue=0.5802072930321989)
MannwhitneyuResult(statistic=2232540.0, pvalue=0.8200440308715571)
MannwhitneyuResult(statistic=1930487.0, pvalue=1.7970498888747568e-08)
MannwhitneyuResult(statistic=2195485.5, pvalue=1.9113726163672255e-07)
